#Task 2 - Trying Different Models

###Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from tensorflow import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

###Importing the datasets (code used from the baseline respository)

In [2]:
#*********************Comment this out and change them appropriately if you wish to run this locally******************************#
path_ptbdb_1 = "/content/drive/MyDrive/ML4HC/archive/ptbdb_normal.csv"
path_ptbdb_2 = "/content/drive/MyDrive/ML4HC/archive/ptbdb_abnormal.csv"

path_mitbih_train = "/content/drive/MyDrive/ML4HC/archive/mitbih_train.csv"
path_mitbih_test = "/content/drive/MyDrive/ML4HC/archive/mitbih_test.csv"

In [3]:
#function needed to reshape input
def convert3Dto2D(X):
  return X.reshape(-1, X.shape[1])

######PTBDB dataset

In [4]:
df_1 = pd.read_csv(path_ptbdb_1, header=None)
df_2 = pd.read_csv(path_ptbdb_2, header=None)

df_ptbdb = pd.concat([df_1, df_2])

df_ptbdb_train, df_ptbdb_test = train_test_split(df_ptbdb, test_size=0.2, random_state=1337, stratify=df_ptbdb[187])

Y_ptbdb_train = np.array(df_ptbdb_train[187].values).astype(np.int8)
X_ptbdb_train = np.array(df_ptbdb_train[list(range(187))].values)[..., np.newaxis]
X_ptbdb_train = convert3Dto2D(X_ptbdb_train)

Y_ptbdb_test = np.array(df_ptbdb_test[187].values).astype(np.int8)
X_ptbdb_test = np.array(df_ptbdb_test[list(range(187))].values)[..., np.newaxis]
X_ptbdb_test = convert3Dto2D(X_ptbdb_test)

######mitbih dataset

In [5]:
df_mitbih_train = pd.read_csv(path_mitbih_train, header=None)
df_mitbih_train = df_mitbih_train.sample(frac=1)
df_mitbih_test = pd.read_csv(path_mitbih_test, header=None)

Y_mitbih_train = np.array(df_mitbih_train[187].values).astype(np.int8)
X_mitbih_train = np.array(df_mitbih_train[list(range(187))].values)[..., np.newaxis]
X_mitbih_train = convert3Dto2D(X_mitbih_train)

Y_mitbih_test = np.array(df_mitbih_test[187].values).astype(np.int8)
X_mitbih_test = np.array(df_mitbih_test[list(range(187))].values)[..., np.newaxis]
X_mitbih_test = convert3Dto2D(X_mitbih_test)

In [6]:
print(np.shape(X_mitbih_train))

(87554, 187)


In [7]:
print(np.shape(X_ptbdb_test))

(2911, 187)


###Create & Train separate models

######Random Forrest

In [8]:
rf_classifier_mitbih = RandomForestClassifier(n_estimators = 1500, max_depth = 15,criterion = 'gini', random_state = 0)
rf_classifier_mitbih.fit(X_mitbih_train, Y_mitbih_train)
rf_classifier_ptbdb = RandomForestClassifier(n_estimators = 1000, max_depth = 12,criterion = 'gini', random_state = 0)
rf_classifier_ptbdb.fit(X_ptbdb_train, Y_ptbdb_train)

RandomForestClassifier(max_depth=12, n_estimators=1000, random_state=0)

#####XGBoost

In [9]:
#initial model
# xgb1 = XGBClassifier(learning_rate=0.1, n_estimators=800,
#                       max_depth=7,
#                       min_child_weight=2,
#                       gamma=0.1,
#                       subsample=0.8,
#                       colsample_bytree=0.8,
#                       nthread=4,
#                       num_class=4,
#                       seed=43)

xgb_classifier_mitbih = XGBClassifier(learning_rate=0.01, n_estimators = 500, max_depth = 6)
xgb_classifier_ptbdb = XGBClassifier(learning_rate=0.01, n_estimators = 500, max_depth = 6)
xgb_classifier_mitbih.fit(X_mitbih_train, Y_mitbih_train)
xgb_classifier_ptbdb.fit(X_ptbdb_train,Y_ptbdb_train)

XGBClassifier(learning_rate=0.01, max_depth=6, n_estimators=500)

######LightGBM

In [10]:
lgb_classifier_mitbih = lgb.LGBMClassifier(learning_rate=0.01,max_depth=6,random_state=0)
lgb_classifier_mitbih.fit(X_mitbih_train,Y_mitbih_train,eval_set=[(X_mitbih_test,Y_mitbih_test),(X_mitbih_train,Y_mitbih_train)],
          verbose=20,eval_metric='logloss')
lgb_classifier_ptbdb = lgb.LGBMClassifier(learning_rate=0.01,max_depth=6,random_state=0)
lgb_classifier_ptbdb.fit(X_ptbdb_train,Y_ptbdb_train,
                         eval_set=[(X_ptbdb_test,Y_ptbdb_test),(X_ptbdb_train,Y_ptbdb_train)],
                         eval_metric='logloss')

[20]	training's multi_logloss: 0.481619	training's multi_logloss: 0.481619	valid_0's multi_logloss: 0.48616	valid_0's multi_logloss: 0.48616
[40]	training's multi_logloss: 0.393844	training's multi_logloss: 0.393844	valid_0's multi_logloss: 0.400599	valid_0's multi_logloss: 0.400599
[60]	training's multi_logloss: 0.334245	training's multi_logloss: 0.334245	valid_0's multi_logloss: 0.342713	valid_0's multi_logloss: 0.342713
[80]	training's multi_logloss: 0.289551	training's multi_logloss: 0.289551	valid_0's multi_logloss: 0.299821	valid_0's multi_logloss: 0.299821
[100]	training's multi_logloss: 0.254983	training's multi_logloss: 0.254983	valid_0's multi_logloss: 0.266863	valid_0's multi_logloss: 0.266863
[20]	training's binary_logloss: 0.503236	training's binary_logloss: 0.503236	valid_0's binary_logloss: 0.508144	valid_0's binary_logloss: 0.508144
[40]	training's binary_logloss: 0.442263	training's binary_logloss: 0.442263	valid_0's binary_logloss: 0.450476	valid_0's binary_logloss: 0

LGBMClassifier(learning_rate=0.01, max_depth=6, random_state=0)

###Model Evaluation

#####Random Forest Evaluation

In [11]:
#mitbih dataset 
Y_mitbih_pred_rf = rf_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_rf, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, rf_classifier_mitbih.predict(X_mitbih_train)))

              precision    recall  f1-score   support

           N     0.9649    0.9993    0.9818     18118
           S     0.9869    0.5432    0.7007       556
           V     0.9822    0.8398    0.9054      1448
           F     0.8488    0.4506    0.5887       162
           Q     0.9947    0.9260    0.9591      1608

    accuracy                         0.9678     21892
   macro avg     0.9555    0.7518    0.8271     21892
weighted avg     0.9679    0.9678    0.9650     21892

-------------------------------
Training set accuracy:  0.9788130753592069


In [12]:
#ptbdb dataset
Y_ptbdb_pred_rf = rf_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_rf, target_names = target_names_ptbdb,digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, rf_classifier_ptbdb.predict(X_ptbdb_train)))

              precision    recall  f1-score   support

          MI     0.9419    0.9023    0.9217       809
           N     0.9630    0.9786    0.9707      2102

    accuracy                         0.9574      2911
   macro avg     0.9525    0.9405    0.9462      2911
weighted avg     0.9572    0.9574    0.9571      2911

-------------------------------
Training set accuracy:  0.9963920625375827


#####XGBoost Evaluation

In [13]:
#mitbih dataset 
Y_mitbih_pred_xg = xgb_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_xg, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, xgb_classifier_mitbih.predict(X_mitbih_train)))


              precision    recall  f1-score   support

           N     0.9689    0.9976    0.9830     18118
           S     0.9623    0.5971    0.7370       556
           V     0.9639    0.8481    0.9023      1448
           F     0.8796    0.5864    0.7037       162
           Q     0.9887    0.9291    0.9580      1608

    accuracy                         0.9694     21892
   macro avg     0.9527    0.7917    0.8568     21892
weighted avg     0.9692    0.9694    0.9675     21892

-------------------------------
Training set accuracy:  0.9765401923384426


In [14]:
#ptbdb dataset
Y_ptbdb_pred_xg = xgb_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_xg, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, xgb_classifier_ptbdb.predict(X_ptbdb_train)))

              precision    recall  f1-score   support

          MI     0.9104    0.9048    0.9076       809
           N     0.9635    0.9657    0.9646      2102

    accuracy                         0.9488      2911
   macro avg     0.9370    0.9353    0.9361      2911
weighted avg     0.9487    0.9488    0.9488      2911

-------------------------------
Training set accuracy:  0.9781805686796667


#####LightGBM Evaluation

In [15]:
#mitbih dataset 
Y_mitbih_pred_lgb = lgb_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_lgb, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, lgb_classifier_mitbih.predict(X_mitbih_train)))

              precision    recall  f1-score   support

           N     0.9374    0.9996    0.9675     18118
           S     0.9858    0.3741    0.5424       556
           V     0.9884    0.6492    0.7837      1448
           F     0.8571    0.1111    0.1967       162
           Q     0.9964    0.8607    0.9236      1608

    accuracy                         0.9438     21892
   macro avg     0.9530    0.5989    0.6828     21892
weighted avg     0.9458    0.9438    0.9356     21892

-------------------------------
Training set accuracy:  0.9452566416154602


In [16]:
#ptbdb dataset
Y_ptbdb_pred_lgb = lgb_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_lgb, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, lgb_classifier_ptbdb.predict(X_ptbdb_train)))

              precision    recall  f1-score   support

          MI     0.8713    0.7194    0.7881       809
           N     0.8988    0.9591    0.9280      2102

    accuracy                         0.8925      2911
   macro avg     0.8850    0.8392    0.8580      2911
weighted avg     0.8911    0.8925    0.8891      2911

-------------------------------
Training set accuracy:  0.9200240529164161


#Task 3 - Ensembles 

######Using the above models we create two VotingClassifiers one based on majority vote and the other one that averages the probabilities of each individual model to make the prediction. This is applied to the ensembles above, thus creating an ensemble of ensembles and then is applied to the CNN and RNN created from Task 1 and 2.

In [ ]:
#mitbih dataset

#soft ensemble - average of probabilities
voting_classifier_mitbih_soft = VotingClassifier([('Random_Forrest', rf_classifier_mitbih),('XGBoost', xgb_classifier_mitbih),('LightGBM',lgb_classifier_mitbih)], 
                 voting = 'soft') #average all predicted probabilities
#hard ensemble - majority vote
voting_classifier_mitbih_hard = VotingClassifier([('Random_Forrest', rf_classifier_mitbih),('XGBoost', xgb_classifier_mitbih),('LightGBM',lgb_classifier_mitbih)], 
                 voting = 'hard') #majority wins principle

voting_classifier_mitbih_soft.fit(X_mitbih_train,Y_mitbih_train) #train the soft model
voting_classifier_mitbih_hard.fit(X_mitbih_train,Y_mitbih_train) #train the hard model

y_mitbih_pred_vc_soft = voting_classifier_mitbih_soft.predict(X_mitbih_test) #make predictions
y_mitbih_pred_vc_hard = voting_classifier_mitbih_hard.predict(X_mitbih_test) #make predictions

print("_______________________________________________________________________________________________________________")
print("Classification report(Soft Classifier): ")
print(classification_report(Y_mitbih_test, y_mitbih_pred_vc_soft, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, voting_classifier_mitbih_soft.predict(X_mitbih_train)))
print("-------------------------------")
print("Test set accuracy(Cross-validation k=5): ", cross_val_score(voting_classifier_mitbih_soft,  X_mitbih_train,Y_mitbih_train).mean())
print("_______________________________________________________________________________________________________________")

print()
print()
print("_______________________________________________________________________________________________________________")
print("Classification report(Hard Classifier): ")
print(classification_report(Y_mitbih_test, y_mitbih_pred_vc_hard, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, voting_classifier_mitbih_hard.predict(X_mitbih_train)))
print("-------------------------------")
print("Test set accuracy(Cross-validation k=5): ", cross_val_score(voting_classifier_mitbih_hard, X_mitbih_train, Y_mitbih_train).mean())
print("_______________________________________________________________________________________________________________")

_______________________________________________________________________________________________________________
Classification report(Soft Classifier): 
              precision    recall  f1-score   support

           N     0.9624    0.9992    0.9805     18118
           S     0.9840    0.5522    0.7074       556
           V     0.9799    0.8101    0.8870      1448
           F     0.8506    0.4568    0.5944       162
           Q     0.9953    0.9192    0.9557      1608

    accuracy                         0.9655     21892
   macro avg     0.9544    0.7475    0.8250     21892
weighted avg     0.9657    0.9655    0.9627     21892

-------------------------------
Training set accuracy:  0.9727368252735454
-------------------------------
Test set accuracy(Cross-validation k=5):  0.9657354311776156
_______________________________________________________________________________________________________________


____________________________________________________________________________

In [ ]:
#ptbdb dataset

#soft ensemble - average of probabilities
voting_classifier_ptbdb_soft = VotingClassifier([('Random_Forrest', rf_classifier_ptbdb),('XGBoost', xgb_classifier_ptbdb),('LightGBM',lgb_classifier_ptbdb)], 
                 voting = 'soft') #average all predicted probabilities
#hard ensemble - majority vote
voting_classifier_ptbdb_hard = VotingClassifier([('Random_Forrest', rf_classifier_ptbdb),('XGBoost', xgb_classifier_ptbdb),('LightGBM',lgb_classifier_ptbdb)], 
                 voting = 'hard') #majority wins principle

voting_classifier_ptbdb_soft.fit(X_ptbdb_train,Y_ptbdb_train) #train the soft model
voting_classifier_ptbdb_hard.fit(X_ptbdb_train,Y_ptbdb_train) #train the hard model

y_ptbdb_pred_vc_soft = voting_classifier_ptbdb_soft.predict(X_ptbdb_test) #make predictions
y_ptbdb_pred_vc_hard = voting_classifier_ptbdb_hard.predict(X_ptbdb_test) #make predictions

print("_______________________________________________________________________________________________________________")
print("Classification report(Soft Classifier): ")
print(classification_report(Y_ptbdb_test, y_ptbdb_pred_vc_soft, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, voting_classifier_ptbdb_soft.predict(X_ptbdb_train)))
print("-------------------------------")
print("Test set accuracy(Cross-validation k=5): ", cross_val_score(voting_classifier_ptbdb_soft,X_ptbdb_train,Y_ptbdb_train).mean())
print("_______________________________________________________________________________________________________________")

print()
print()
print("_______________________________________________________________________________________________________________")
print("Classification report(Hard Classifier): ")
print(classification_report(Y_ptbdb_test, y_ptbdb_pred_vc_hard, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, voting_classifier_ptbdb_hard.predict(X_ptbdb_train)))
print("-------------------------------")
print("Test set accuracy(Cross-validation k=5): ", cross_val_score(voting_classifier_ptbdb_hard,X_ptbdb_train,Y_ptbdb_train).mean())
print("_______________________________________________________________________________________________________________")